In [40]:
import time
import pandas as pd
import numpy as np
from thefuzz import process # had to conda install; to do string comparisons
import recordlinkage # had to conda install; to link disparate records in datasets


In [2]:
poker_path = "data/poker_hand.csv"

In [6]:
loan_path = "data/banking_dirty.csv"
loan_df = pd.read_csv(loan_path)
loan_df.head()

,Unnamed: 0,cust_id,birth_date,Age,acct_amount,inv_amount,fund_A,fund_B,fund_C,fund_D,account_opened,last_transaction
0,0,870A9281,1962-06-09,58,63523.31,51295,30105.0,4138.0,1420.0,15632.0,02-09-18,22-02-19
1,1,166B05B0,1962-12-16,58,38175.46,15050,4995.0,938.0,6696.0,2421.0,28-02-19,31-10-18
2,2,BFC13E88,1990-09-12,34,59863.77,24567,10323.0,4590.0,8469.0,1185.0,25-04-18,02-04-18
3,3,F2158F66,1985-11-03,35,84132.10,23712,3908.0,492.0,6482.0,12830.0,07-11-17,08-11-18
4,4,7A73F334,1990-05-17,30,120512.00,93230,12158.4,51281.0,13434.0,18383.0,14-05-18,19-07-18


In [17]:
# If you don't put "keep=False," it only returns one of the dupes
duplicates = loan_df.duplicated(subset = 'birth_date', keep = False)
loan_df[duplicates]

,Unnamed: 0,cust_id,birth_date,Age,acct_amount,inv_amount,fund_A,fund_B,fund_C,fund_D,account_opened,last_transaction
8,8,E52D4C7F,1975-06-05,49,61795.89,49385,12939.0,7757.0,12569.0,16120.0,22-05-17,24-10-19
45,45,7A4EED75,1975-06-05,45,98923.14,20931,10107.0,393.0,423.0,10008.0,28-11-17,01-05-19


In [18]:
temp_df = pd.read_csv("data/temperatures.csv")

In [19]:
temp_df.head()

,date,city,country,avg_temp_c
0,2000-01-01,Abidjan,Côte D'Ivoire,27.293
1,2000-02-01,Abidjan,Côte D'Ivoire,27.685
2,2000-03-01,Abidjan,Côte D'Ivoire,29.061
3,2000-04-01,Abidjan,Côte D'Ivoire,28.162
4,2000-05-01,Abidjan,Côte D'Ivoire,27.547


In [23]:
# This second parameter JUST takes that column
temp_df.loc[temp_df['avg_temp_c'] > 30, 'avg_temp_c'] 

333      32.275
334      32.847
335      32.490
345      31.034
346      33.358
          ...  
16212    30.141
16236    30.061
16320    30.605
16332    31.171
16333    31.050
Name: avg_temp_c, Length: 1156, dtype: float64

In [29]:
# These two methods are basically equal
temp_df[temp_df['avg_temp_c'] > 30] == temp_df.loc[temp_df['avg_temp_c'] > 30]

,date,city,country,avg_temp_c
333,True,True,True,True
334,True,True,True,True
335,True,True,True,True
345,True,True,True,True
346,True,True,True,True
...,...,...,...,...
16212,True,True,True,True
16236,True,True,True,True
16320,True,True,True,True
16332,True,True,True,True


In [26]:
banking = pd.read_csv("data/banking_dirty.csv")

In [28]:
banking.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        100 non-null    int64  
 1   cust_id           100 non-null    object 
 2   birth_date        100 non-null    object 
 3   Age               100 non-null    int64  
 4   acct_amount       100 non-null    float64
 5   inv_amount        100 non-null    int64  
 6   fund_A            100 non-null    float64
 7   fund_B            100 non-null    float64
 8   fund_C            100 non-null    float64
 9   fund_D            100 non-null    float64
 10  account_opened    100 non-null    object 
 11  last_transaction  100 non-null    object 
dtypes: float64(5), int64(3), object(4)
memory usage: 9.5+ KB


In [32]:
restaurants = pd.read_csv("data/restaurants_L2_dirty.csv")

In [33]:
restaurants.head(
)

,Unnamed: 0,name,addr,city,phone,type
0,0,kokomo,6333 w. third st.,la,2139330773,american
1,1,feenix,8358 sunset blvd. west,hollywood,2138486677,american
2,2,parkway,510 s. arroyo pkwy .,pasadena,8187951001,californian
3,3,r-23,923 e. third st.,los angeles,2136877178,japanese
4,4,gumbo,6333 w. third st.,la,2139330358,cajun/creole


In [34]:
unique_type = restaurants['type'].unique()

In [35]:
unique_type

array(['american', 'californian', 'japanese', 'cajun/creole', 'hot dogs',
       'diners', 'delis', 'hamburgers', 'seafood', 'italian',
       'coffee shops', 'russian', 'steakhouses', 'mexican/tex-mex',
       'noodle shops', 'mexican', 'middle eastern', 'asian', 'vietnamese',
       'health food', 'american ( new )', 'pacific new wave',
       'indonesian', 'eclectic', 'chicken', 'fast food', 'southern/soul',
       'coffeebar', 'continental', 'french ( new )', 'desserts',
       'chinese', 'pizza'], dtype=object)

In [38]:
matches = process.extract(
    'italian', restaurants['type'], limit=len(restaurants.type))
print(matches)

[('italian', 100, 14), ('italian', 100, 21), ('italian', 100, 47), ('italian', 100, 57), ('italian', 100, 73), ('italian', 100, 75), ('italian', 100, 76), ('asian', 67, 26), ('asian', 67, 65), ('asian', 67, 71), ('asian', 67, 79), ('californian', 56, 2), ('californian', 56, 30), ('californian', 56, 34), ('californian', 56, 36), ('californian', 56, 80), ('american', 53, 0), ('american', 53, 1), ('american', 53, 7), ('american', 53, 12), ('american', 53, 13), ('american', 53, 20), ('american', 53, 27), ('american', 53, 28), ('american', 53, 39), ('american', 53, 40), ('american', 53, 43), ('american', 53, 49), ('american', 53, 50), ('american', 53, 51), ('american', 53, 53), ('american', 53, 54), ('american', 53, 61), ('american', 53, 67), ('american', 53, 74), ('american', 53, 81), ('continental', 51, 60), ('indonesian', 47, 41), ('russian', 43, 16), ('mexican', 43, 24), ('mexican', 43, 63), ('mexican', 43, 66), ('japanese', 40, 3), ('mexican/tex-mex', 39, 19), ('american ( new )', 39, 